In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import random
import re

from tqdm import tqdm
import time

import pydicom as dicom
import nibabel as nib
import SimpleITK as sitk
import monai

import torch
import torch.nn as nn
import torch.optim as optim

from monai.networks.nets import EfficientNetBN
from monai.networks.nets import ResNet
#from efficientnet_pytorch import EfficientNet
import timm

import wandb


In [2]:
SEED = 344
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True # Fix the network according to random seed
    print('Finish seeding with seed {}'.format(seed))
    
seed_everything(SEED)
print('Training on device {}'.format(device))

Finish seeding with seed 344
Training on device cuda


In [3]:
dicom_tag_columns = [
    'Columns',
    'ImageOrientationPatient',
    'ImagePositionPatient',
    'InstanceNumber',
    'PatientID',
    'PatientPosition',
    'PixelSpacing',
    'RescaleIntercept',
    'RescaleSlope',
    'Rows',
    'SeriesNumber',
    'SliceThickness',
    'path',
    'WindowCenter',
    'WindowWidth'
]

train_dicom_tags = pd.read_parquet('autodl-tmp/train_dicom_tags.parquet', columns=dicom_tag_columns)
test_dicom_tags = pd.read_parquet('autodl-tmp/test_dicom_tags.parquet', columns=dicom_tag_columns)

train_series_meta = pd.read_csv('autodl-tmp/train_series_meta.csv')
test_series_meta = pd.read_csv('autodl-tmp/test_series_meta.csv')

train_csv = pd.read_csv('autodl-tmp/train.csv')

fold1_2phase_csv = pd.read_csv('autodl-tmp/train_3fold/fold1_2phase.csv')
fold2_2phase_csv = pd.read_csv('autodl-tmp/train_3fold/fold2_2phase.csv')
fold3_2phase_csv = pd.read_csv('autodl-tmp/train_3fold/fold3_2phase.csv')

fold3_2phase_csv

,index,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury,series_num
0,2091,4905,1,0,1,0,1,0,0,1,0,0,1,0,0,0,1
1,2092,49075,1,0,1,0,1,0,0,1,0,0,1,0,0,0,1
2,2095,49134,1,0,1,0,1,0,0,1,0,0,1,0,0,0,1
3,2097,49145,1,0,1,0,1,0,0,1,0,0,1,0,0,0,1
4,2099,49167,1,0,1,0,0,1,0,1,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,3137,9801,1,0,1,0,1,0,0,1,0,0,1,0,0,0,1
508,3138,9808,1,0,1,0,1,0,0,1,0,0,1,0,0,0,1
509,3140,9835,1,0,1,0,1,0,0,1,0,0,1,0,0,0,1
510,3144,9961,1,0,1,0,1,0,0,1,0,0,1,0,0,0,1


In [4]:
injury_series_meta = train_series_meta.loc[train_series_meta.patient_id.isin(train_csv.loc[train_csv.any_injury == 1, "patient_id"].values)]
healthy_series_meta = train_series_meta.loc[train_series_meta.patient_id.isin(train_csv.loc[train_csv.any_injury == 0, "patient_id"].values)]

In [5]:
def raw_path_gen(patient_id, series_id, train=True):
    if(train):
        path = 'autodl-tmp/train_images_resample/'
    else:
        path = 'autodl-tmp/train_images_resample/'
    
    path += str(patient_id) + '/' + str(series_id)
    
    return path

def create_3D_scans(folder, downsample_rate=1): 
    filenames = os.listdir(folder)
    filenames = [int(filename.split('.')[0]) for filename in filenames]
    filenames = sorted(filenames)
    filenames = [str(filename) + '.dcm' for filename in filenames]
        
    volume = []
    #for filename in tqdm(filenames[::downsample_rate], position=0): 
    for filename in filenames[::downsample_rate]: 
        filepath = os.path.join(folder, filename)
        ds = dicom.dcmread(filepath)
        image = ds.pixel_array
        
        if ds.PixelRepresentation == 1:
            bit_shift = ds.BitsAllocated - ds.BitsStored
            dtype = image.dtype 
            image = (image << bit_shift).astype(dtype) >>  bit_shift
        
        # find rescale params
        if ("RescaleIntercept" in ds) and ("RescaleSlope" in ds):
            intercept = float(ds.RescaleIntercept)
            slope = float(ds.RescaleSlope)
    
        # find clipping params
        center = int(ds.WindowCenter)
        width = int(ds.WindowWidth)
        low = center - width / 2
        high = center + width / 2    
        
        
        image = (image * slope) + intercept
        image = np.clip(image, low, high)

        image = (image / np.max(image) * 255).astype(np.int16)
        image = image[::downsample_rate, ::downsample_rate]
        volume.append( image )
    
    volume = np.stack(volume, axis=0)
    return volume

def plot_image_with_seg(volume, volume_seg=[], orientation='Coronal', num_subplots=20):
    # simply copy
    if len(volume_seg) == 0:
        plot_mask = 0
    else:
        plot_mask = 1
        
    if orientation == 'Coronal':
        slices = np.linspace(0, volume.shape[2]-1, num_subplots).astype(np.int16)
        volume = volume.transpose([1, 0, 2])
        if plot_mask:
            volume_seg = volume_seg.transpose([1, 0, 2])
        
    elif orientation == 'Sagittal':
        slices = np.linspace(0, volume.shape[2]-1, num_subplots).astype(np.int16)
        volume = volume.transpose([2, 0, 1])
        if plot_mask:
            volume_seg = volume_seg.transpose([2, 0, 1])

    elif orientation == 'Axial':
        slices = np.linspace(0, volume.shape[0]-1, num_subplots).astype(np.int16)
           
    rows = np.max( [np.floor(np.sqrt(num_subplots)).astype(int) - 2, 1])
    cols = np.ceil(num_subplots/rows).astype(int)
    
    fig, ax = plt.subplots(rows, cols, figsize=(cols * 2, rows * 4))
    fig.tight_layout(h_pad=0.01, w_pad=0)
    
    ax = ax.ravel()
    for this_ax in ax:
        this_ax.axis('off')

    for counter, this_slice in enumerate( slices ):
        plt.sca(ax[counter])
        
        image = volume[this_slice, :, :]
        plt.imshow(image, cmap='gray')
        
        if plot_mask:
            mask = np.where(volume_seg[this_slice, :, :], volume_seg[this_slice, :, :], np.nan)
            plt.imshow(mask, cmap='Set1', alpha=0.5)
            
def load_nii(patient_id, series_id, root='autodl-tmp/train_images_resample/'):
    path = root + str(patient_id) + '/' + str(series_id) + '.nii.gz'
    img = sitk.ReadImage(path)
    img = sitk.GetArrayFromImage(img)
    
    # img = nib.load(path)
    # img = img.get_fdata().transpose(2, 1, 0)
    
    return img

In [6]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

ds = 2

class CTDataset_2phase(Dataset):
    def __init__(self, root='autodl-tmp/train_images_resample/', augmentation=False, subjects=train_csv, meta=train_series_meta, device='cpu'):
        self.device = device
        self.series_meta = meta
        self.subjects_csv = subjects
        self.root = root
        self.t = monai.transforms.Compose([monai.transforms.RandZoom(prob=0.5, min_zoom=0.9, max_zoom=1.1),
                                           monai.transforms.RandRotate(range_x=3.14 / 24, prob=0.5),
                                           monai.transforms.SpatialPad(spatial_size=(320//ds, 280//ds, 280//ds), mode="edge"),
                                           monai.transforms.RandSpatialCrop(roi_size=(320//ds, 256//ds, 256//ds), random_size=False),
                                           monai.transforms.NormalizeIntensity(divisor = 400)
                                ])
        self.t_val = monai.transforms.Compose([monai.transforms.NormalizeIntensity(divisor = 400)])
        
        self.aug = augmentation
        
    def __len__(self):
        #return 1100
        return len(self.subjects_csv)
    
    def __getitem__(self, idx): # high hu first
        
        patient_id = self.subjects_csv.loc[idx, ['patient_id']].values[0].astype('int')
        series_id_1, hu1 = self.series_meta.loc[self.series_meta.patient_id == patient_id, ['series_id', 'aortic_hu']].values[0].astype('int')
        series_id_2, hu2 = self.series_meta.loc[self.series_meta.patient_id == patient_id, ['series_id', 'aortic_hu']].values[1].astype('int')
        
        if hu1 > hu2:
            series_id_h = series_id_1
            series_id_l = series_id_2
        else:
            series_id_h = series_id_2
            series_id_l = series_id_1
        
        
        img_a_h = load_nii(patient_id, series_id_h, self.root).astype('float32')
        if(self.aug):
            img_t_h = self.t(np.expand_dims(img_a_h[::ds, ::ds, ::ds], 0))
        else:
            img_t_h = self.t_val(np.expand_dims(img_a_h[::ds, ::ds, ::ds], 0))
            
        img_a_l = load_nii(patient_id, series_id_l, self.root).astype('float32')
        if(self.aug):
            img_t_l = self.t(np.expand_dims(img_a_l[::ds, ::ds, ::ds], 0))
        else:
            img_t_l = self.t_val(np.expand_dims(img_a_l[::ds, ::ds, ::ds], 0))
            
        label_columns = [
            'bowel_injury',
            'extravasation_injury',
            'kidney_low',
            'kidney_high',
            'liver_low',
            'liver_high',
            'spleen_low',
            'spleen_high',
            #'any_injury'
        ]
        label_a = self.subjects_csv.loc[idx, label_columns].values.astype('float32')
        label_t = torch.from_numpy(label_a)
        return img_t_h, img_t_l, label_t
        

        # patient_id, series_id = self.series_meta.loc[idx, ["patient_id", "series_id"]].astype('int')
        # img_a = load_nii(patient_id, series_id, self.root).astype('float32')
        # #img_t = torch.from_numpy(img_a).unsqueeze(0)
        # #img_t = torch.from_numpy(img_a[::2, ::2, ::2]).unsqueeze(0)
        # if(self.aug):
        #     img_t = self.t(np.expand_dims(img_a[::ds, ::ds, ::ds], 0))
        # else:
        #     #img_t = torch.from_numpy(img_a[::ds, ::ds, ::ds]).unsqueeze(0)
        #     img_t = self.t_val(np.expand_dims(img_a[::ds, ::ds, ::ds], 0))
        # label_columns = [
        #     'bowel_injury',
        #     'extravasation_injury',
        #     'kidney_low',
        #     'kidney_high',
        #     'liver_low',
        #     'liver_high',
        #     'spleen_low',
        #     'spleen_high',
        #     #'any_injury'
        # ]
        # label_a = train_csv.loc[train_csv.patient_id == patient_id, label_columns].values[0].astype('float32')
        # label_t = torch.from_numpy(label_a)
        # return img_t, label_t

In [7]:
# train_meta = train_series_meta[-3600:].reset_index()
# val_meta = train_series_meta[0:-3600].reset_index()

# train_ds = CTDataset(meta = train_meta, augmentation=True)
# train_dl = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=8)

# val_ds = CTDataset(meta = val_meta, augmentation=False)
# val_dl = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=8)

In [8]:
# class EffNet(nn.Module):
#     def __init__(self, ch_out=9):
#         super(EffNet, self).__init__()
#         #self.conv_in = nn.Conv3d(1, 3, kernel_size=5, padding=2, stride=2)
#         self.net = EfficientNetBN("efficientnet-b0", pretrained=False, progress=False, spatial_dims=3, in_channels=1, num_classes=ch_out,)
#     def forward(self, x):
#         #x = self.conv_in(x)
#         #return torch.sigmoid(self.net(x))
#         return self.net(x)

class FullVolNet_2phase(nn.Module):
    def __init__(self, backbone = "tf_efficientnetv2_s.in21k_ft_in1k", 
                 ch_in = 3, ch_out = 9, slices = 15, dropout = 0.0, pretrained=True):
        super(FullVolNet_2phase, self).__init__()
        self.slices = slices
        
        self.encoder1 = timm.create_model(
            backbone,
            in_chans=ch_in,
            num_classes=ch_out,
            features_only=False,
            drop_rate=0.0,
            drop_path_rate=0.0,
            pretrained=False,
        )
        
        self.encoder2 = timm.create_model(
            backbone,
            in_chans=ch_in,
            num_classes=ch_out,
            features_only=False,
            drop_rate=0.0,
            drop_path_rate=0.0,
            pretrained=False,
        )
        
        if 'efficient' in backbone and pretrained:
            self.encoder1.load_state_dict(torch.load('pretrained/tf_efficientnetv2_s.in21k_ft_in1k_8class.pt'))
            self.encoder2.load_state_dict(torch.load('pretrained/tf_efficientnetv2_s.in21k_ft_in1k_8class.pt'))
        elif 'convnext' in backbone and pretrained:
            self.encoder1.load_state_dict(torch.load('pretrained/convnextv2_nano.fcmae_ft_in22k_in1k_384_8class.pt'))
            self.encoder2.load_state_dict(torch.load('pretrained/convnextv2_nano.fcmae_ft_in22k_in1k_384_8class.pt'))
        
        if 'efficient' in backbone:
            hdim = self.encoder1.conv_head.out_channels
            self.encoder1.classifier = nn.Identity()
            self.encoder2.classifier = nn.Identity()
        elif 'convnext' in backbone:
            hdim = self.encoder1.head.fc.in_features
            self.encoder1.head.fc = nn.Identity()
            self.encoder2.head.fc = nn.Identity()
        
        self.lstm = nn.LSTM(hdim, 256, num_layers=2, dropout=0.0, bidirectional=True, batch_first=True)
        
        self.head = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.Dropout(dropout),
            nn.LeakyReLU(0.1),
            nn.Linear(256, ch_out),
        )
        
        self.head2 = nn.Conv1d(2*slices, 1, 1)
        
        
    def slicer(self, img, slices): 
        #img = img.squeeze(1)
        z_length = img.shape[-3]
        z_slices = (np.linspace(0, z_length, slices + 4)).astype('int')
        z_slices = z_slices[2:-2]
        #print(z_slices)
        slices_list = []
        for z in z_slices:
            slices_list.append(img[:, :, z-1:z+2, :, :])
        img_slice = torch.cat(slices_list, 1)
        return img_slice
        
    def forward(self, x1, x2):  # (bs, nslice, ch, sz, sz) high hu first
        x1 = self.slicer(x1, self.slices)
        x2 = self.slicer(x2, self.slices)
        bs, nslice,ch, sz1, sz2 = x1.shape
        x1 = x1.view(bs*nslice, ch, sz1, sz2)
        x2 = x2.view(bs*nslice, ch, sz1, sz2)
        
        feature_2d_1 = self.encoder1(x1)
        feature_2d_1 = feature_2d_1.view(bs, nslice, -1)
        feature_2d_2 = self.encoder2(x2)
        feature_2d_2 = feature_2d_2.view(bs, nslice, -1)
        
        feature_2d = torch.cat([feature_2d_1, feature_2d_2], axis=1)
        
        feature_lstm, _ = self.lstm(feature_2d)
        feature_lstm = feature_lstm.contiguous().view(bs * nslice * 2, -1)
        
        preds = self.head(feature_lstm)
        preds = preds.view(bs, nslice * 2, -1).contiguous()
        preds = self.head2(preds)
        
        return preds.squeeze(1)
        
        
        # bs = x.shape[0]
        # x = x.view(bs * n_slice_per_c, in_chans, image_size, image_size)
        # feat = self.encoder(x)
        # feat = feat.view(bs, n_slice_per_c, -1)
        # feat, _ = self.lstm(feat)
        # feat = feat.contiguous().view(bs * n_slice_per_c, -1)
        # feat = self.head(feat)
        # feat = feat.view(bs, n_slice_per_c).contiguous()
        

In [9]:
# imgs, labels = next(iter(train_dl))
# net = FullVolNet(ch_out = 13)
# img_s = net(imgs)
# img_s.shape

In [10]:
import sklearn.metrics

def transform_9class(label_in):
    label_out = [1 - label_in[0],
                   label_in[0],
                    1- label_in[1],
                   label_in[1],
                   (1 - label_in[2]) * (1 - label_in[3]),
                   label_in[2],
                   label_in[3],
                   (1 - label_in[4]) * (1 - label_in[5]),
                    label_in[4],
                   label_in[5],
                   (1 - label_in[6]) * (1 - label_in[7]),
                   label_in[6],
                   label_in[7]]
    return label_out

def transform_13class(label_in):
    label_out = label_in
    return label_out.tolist()


def loss_metrics(metrics, transform):
    preds = [transform(x) for x in metrics["predict"]]
    targets = [transform(x) for x in metrics["label"]]
    targets_any_injury = metrics["label"][:, -1]
    
    loss_list = []
    
    print("F1 score: ", sklearn.metrics.f1_score(metrics["label"], np.around(metrics["predict"]), average=None, zero_division=0.0))
    print("AUC score: ", sklearn.metrics.roc_auc_score(metrics["label"], metrics["predict"], average=None))
    
    for i in range(0, len(preds)):
        predict = preds[i]
        target = targets[i]
        
        label_pred = np.zeros(14)
        label_pred[0] = predict[0] / (predict[0] + predict[1])
        label_pred[1] = predict[1] / (predict[0] + predict[1])
        label_pred[2] = predict[2] / (predict[2] + predict[3])
        label_pred[3] = predict[3] / (predict[2] + predict[3])
        label_pred[4] = predict[4] / (predict[4] + predict[5] + predict[6])
        label_pred[5] = predict[5] / (predict[4] + predict[5] + predict[6])
        label_pred[6] = predict[6] / (predict[4] + predict[5] + predict[6])
        label_pred[7] = predict[7] / (predict[7] + predict[8] + predict[9])
        label_pred[8] = predict[8] / (predict[7] + predict[8] + predict[9])
        label_pred[9] = predict[9] / (predict[7] + predict[8] + predict[9])
        label_pred[10] = predict[10] / (predict[10] + predict[11] + predict[12])
        label_pred[11] = predict[11] / (predict[10] + predict[11] + predict[12])
        label_pred[12] = predict[12] / (predict[10] + predict[11] + predict[12])
        label_pred[13] = max([1 - label_pred[x] for x in [0, 2, 4, 7, 10]])
        
        targets_any_injury = max([1 - target[x] for x in [0, 2, 4, 7, 10]])
        
        target.append(targets_any_injury)
        label_target = np.array(target)
        
        weight = np.array([1, 2, 1, 6, 1, 2, 4, 1, 2, 4, 1, 2, 4, 6])
        
        loss_list.append(sklearn.metrics.log_loss(
            y_true=label_target,
            y_pred=label_pred,
            sample_weight=weight))
    #print("Weighted Loss: " + np.mean(loss_list))
    
    return np.mean(loss_list)
        
    
    
    #print(np.array(preds).shape)
    

In [11]:
import copy

def TrainClassifer(model,trn_dl,val_dl,optimizer, project, name, suffix, scheduler=None,
                   n_eopchs=20, device='cpu'):
 
    #loss_fn = nn.BCELoss(weight=torch.Tensor([1, 6, 1, 6, 1, 4, 8, 1, 4, 8, 1, 4, 8]).to(device))
    #loss_fn = nn.BCELoss()
    loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([10, 5, 10, 10, 6, 6, 6, 6]).to(device))
    model.to(device)
    best_model = copy.deepcopy(model)
    bestweight_model = copy.deepcopy(model)
    best_val = 999.0
    best_weightloss = 999.0
    metrics = {'predict': [], 'label' : []}
    PATH_MODEL = project + '/' + name + '/' + suffix + '.pt'
    wandb.init(name=name, 
               project=project)

    for epoch in range(1, n_eopchs + 1):
        loss_train = 0.0
        model.train()
        for imgs_h, imgs_l, labels in tqdm(trn_dl, position=0):
            imgs_h = imgs_h.to(device)
            imgs_l = imgs_l.to(device)
            labels = labels.to(device)

            outputs = model(imgs_h, imgs_l)
            #outputs = model(imgs.unsqueeze(1))
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
        torch.cuda.empty_cache()
        
        loss_val = 0.0
        correct_val = 0.0
        model.eval()
        
        for imgs_h, imgs_l, labels in tqdm(val_dl):
            imgs_h = imgs_h.to(device)
            imgs_l = imgs_l.to(device)
            labels = labels.to(device)
            with torch.no_grad():
                outputs = model(imgs_h, imgs_l)
                #outputs = model(imgs.unsqueeze(1))
                loss = loss_fn(outputs, labels)
                loss_val += loss.item()
                outputs = torch.sigmoid(outputs)
                metrics['predict'].extend((outputs.to('cpu').detach().numpy()).tolist())
                metrics['label'].extend((labels.to('cpu').detach().numpy()).tolist())
        
        metrics['predict'] = np.array(metrics['predict'])
        #metrics['predict'] = np.array([[0.5, 0.5, 0.33333, 0.33333, 0.33333, 0.33333, 0.33333, 0.33333, 0.33333]]*len(metrics['label']))
        metrics['label'] = np.array(metrics['label'])
        weighted_loss = loss_metrics(metrics, transform_9class)
        metrics = {'predict': [], 'label' : []}
        
        torch.cuda.empty_cache()
        
        if (weighted_loss) < best_weightloss:
            best_weightloss = weighted_loss
            
        if loss_val / len(val_dl) < best_val:
            best_val = loss_val / len(val_dl)
            torch.save(model.state_dict(), 'model_tmp.pt')
            best_model.load_state_dict(torch.load('model_tmp.pt'))
            
            
        if scheduler != None:
            scheduler.step()

        print('{} Eopch {}, Training Loss {}, Val Loss {}, Weighted Loss {}'.format(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()),
                                                                  epoch, loss_train / len(trn_dl), loss_val / len(val_dl),
                                                                                     weighted_loss))
        
        
        
        wandb.log({'training loss': loss_train / len(trn_dl),
                  'val loss': loss_val / len(val_dl),
                  'weighted loss': weighted_loss})
    torch.save(best_model.state_dict(), PATH_MODEL)
    print('Finish training: best_val:{}, best_weighted loss:{}'.format(best_val, best_weightloss))
    wandb.finish()

In [12]:
import os

project = "FullVol2.5D-3fold"
name = "FullVol2.5D-2phase_Effnetv2_3fold"
backbone = "tf_efficientnetv2_s.in21k_ft_in1k"
# backbone = 'convnextv2_nano.fcmae_ft_in22k_in1k_384'
n_eopchs = 20

p = project + '/' + name
if not os.path.exists(p):
    os.mkdir(p)

In [13]:
# fold1_1phase_csv = pd.read_csv('autodl-tmp/train_3fold/fold1_1phase.csv')
# fold2_1phase_csv = pd.read_csv('autodl-tmp/train_3fold/fold2_1phase.csv')
# fold3_1phase_csv = pd.read_csv('autodl-tmp/train_3fold/fold3_1phase.csv')

train_df_fold1 = pd.concat([fold1_2phase_csv, fold2_2phase_csv], axis=0).reset_index()
train_df_fold2 = pd.concat([fold1_2phase_csv, fold3_2phase_csv], axis=0).reset_index()
train_df_fold3 = pd.concat([fold2_2phase_csv, fold3_2phase_csv], axis=0).reset_index()

val_df_fold1 = fold3_2phase_csv
val_df_fold2 = fold2_2phase_csv
val_df_fold3 = fold1_2phase_csv

train_ds_fold1 = CTDataset_2phase(subjects = train_df_fold1, meta = train_series_meta, augmentation=True)
train_dl_fold1 = DataLoader(train_ds_fold1, batch_size=4, shuffle=True, num_workers=8)
val_ds_fold1 = CTDataset_2phase(subjects = val_df_fold1, meta = train_series_meta, augmentation=False)
val_dl_fold1 = DataLoader(val_ds_fold1, batch_size=4, shuffle=False, num_workers=8)

train_ds_fold2 = CTDataset_2phase(subjects = train_df_fold2, meta = train_series_meta, augmentation=True)
train_dl_fold2 = DataLoader(train_ds_fold2, batch_size=4, shuffle=True, num_workers=8)
val_ds_fold2 = CTDataset_2phase(subjects = val_df_fold2, meta = train_series_meta, augmentation=False)
val_dl_fold2 = DataLoader(val_ds_fold2, batch_size=4, shuffle=False, num_workers=8)

train_ds_fold3 = CTDataset_2phase(subjects = train_df_fold3, meta = train_series_meta, augmentation=True)
train_dl_fold3 = DataLoader(train_ds_fold3, batch_size=4, shuffle=True, num_workers=8)
val_ds_fold3 = CTDataset_2phase(subjects = val_df_fold3, meta = train_series_meta, augmentation=False)
val_dl_fold3 = DataLoader(val_ds_fold3, batch_size=4, shuffle=False, num_workers=8)

In [14]:
net = FullVolNet_2phase(backbone=backbone, ch_out = 8).to(device)
optimizer = optim.AdamW(net.parameters(), lr=1e-5)
TrainClassifer(model=net,trn_dl=train_dl_fold1,val_dl=val_dl_fold1,optimizer=optimizer, 
               project=project, name=name, suffix='fold1', scheduler=None, n_eopchs=n_eopchs, device=device)

net = FullVolNet_2phase(backbone=backbone, ch_out = 8).to(device)
optimizer = optim.AdamW(net.parameters(), lr=1e-5)
TrainClassifer(model=net,trn_dl=train_dl_fold2,val_dl=val_dl_fold2,optimizer=optimizer, 
               project=project, name=name, suffix='fold2', scheduler=None, n_eopchs=n_eopchs, device=device)

net = FullVolNet_2phase(backbone=backbone, ch_out = 8).to(device)
optimizer = optim.AdamW(net.parameters(), lr=1e-5)
TrainClassifer(model=net,trn_dl=train_dl_fold3,val_dl=val_dl_fold3,optimizer=optimizer, 
               project=project, name=name, suffix='fold3', scheduler=None, n_eopchs=n_eopchs, device=device)

wandb: Currently logged in as: northm (rsna2023). Use `wandb login --relogin` to force relogin


100%|██████████| 128/128 [01:16<00:00,  1.66it/s]


F1 score:  [0.         0.05405405 0.         0.         0.09836066 0.
 0.12121212 0.        ]
AUC score:  [0.67900738 0.6235242  0.62804878 0.52596554 0.54685765 0.53695437
 0.56068737 0.54081633]
2023-09-28 06:04:07 Eopch 1, Training Loss 0.7940957458980183, Val Loss 0.7164111116435379, Weighted Loss 0.5076543100240152


100%|██████████| 128/128 [01:07<00:00,  1.90it/s]


F1 score:  [0.         0.06451613 0.         0.         0.11320755 0.
 0.08       0.        ]
AUC score:  [0.70046948 0.66263282 0.61737805 0.53048128 0.59105041 0.50892857
 0.58101588 0.55565863]
2023-09-28 06:08:19 Eopch 2, Training Loss 0.6944076723698881, Val Loss 0.6888369249645621, Weighted Loss 0.4229421280817416


...

100%|██████████| 132/132 [01:27<00:00,  1.50it/s]


F1 score:  [0.         0.06153846 0.14545455 0.20689655 0.15254237 0.
 0.19565217 0.13953488]
AUC score:  [0.79970127 0.65046296 0.62103175 0.6137416  0.61816578 0.71107881
 0.63923182 0.67499006]
2023-09-28 10:04:57 Eopch 19, Training Loss 0.39057898112935907, Val Loss 0.7538710215100737, Weighted Loss 0.3153516943362846


100%|██████████| 132/132 [01:09<00:00,  1.91it/s]


F1 score:  [0.         0.07142857 0.16949153 0.13333333 0.13333333 0.
 0.18421053 0.        ]
AUC score:  [0.79507095 0.6732159  0.62136243 0.63181479 0.62541642 0.6873385
 0.64643347 0.67395626]
2023-09-28 10:09:01 Eopch 20, Training Loss 0.3673075426614423, Val Loss 0.7896336408783541, Weighted Loss 0.29410881518220666
Finish training: best_val:0.6759402940444874, best_weighted loss:0.2857122292664789


training loss,█▇▆▆▆▅▅▅▄▄▄▄▃▃▂▂▂▂▁▁
val loss,▆▄▃▂▂▂▁▁▁▂▂▁▂▁▂▅▆▅▆█
weighted loss,█▆▅▄▃▃▃▃▃▂▂▂▂▂▃▁▁▂▂▁
training loss,0.36731
val loss,0.78963
weighted loss,0.29411
